# CSC-52081-EP Lab7: Optimization - CEM / ES

<img src="https://raw.githubusercontent.com/jeremiedecock/polytechnique-csc-52081-ep-2025-students/refs/heads/main/assets/logo.jpg" style="float: left; width: 15%" />

[CSC-52081-EP-2025](https://moodle.polytechnique.fr/course/view.php?id=19336) Lab session #7

2019-2025 Jérémie Decock

[![Open in Google Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/jeremiedecock/polytechnique-csc-52081-ep-2025-students/blob/main/lab7_optim.ipynb)

[![My Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/jeremiedecock/polytechnique-csc-52081-ep-2025-students/main?filepath=lab7_optim.ipynb)

[![NbViewer](https://raw.githubusercontent.com/jupyter/design/main/logos/Badges/nbviewer_badge.svg)](https://nbviewer.jupyter.org/github/jeremiedecock/polytechnique-csc-52081-ep-2025-students/blob/main/lab7_optim.ipynb)

[![Local](https://img.shields.io/badge/Local-Save%20As...-blue)](https://github.com/jeremiedecock/polytechnique-csc-52081-ep-2025-students/raw/main/lab7_optim.ipynb)

## Introduction

In the previous lab we studied a method that allowed us to apply reinforcement learning in continuous state spaces and/or continuous action spaces.
We used REINFORCE, a *Policy gradient* method that directly optimize the parametric policy $\pi_{\theta}$.
The parameter $\theta$ was iteratively updated toward a local maximum of the total expected reward $J(\theta)$ using a gradient ascent method:
$$\theta \leftarrow \theta + \alpha \nabla_{\theta}J(\theta)$$
A convenient analytical formulation of $\nabla_{\theta}J(\theta)$ was obtained thanks to the *Policy Gradient theorem*:

$$\nabla_\theta J(\theta) = \nabla_\theta V^{\pi_\theta}(s) = \mathbb{E}_{\pi_\theta} \left[\nabla_\theta \log \pi_\theta (a|s) Q^{\pi_\theta}(s,a) \right].$$
However, gradient ascent methods may have a slow convergence and will only found a local optimum.
Moreover, this approach requires an analytical formulation of $\nabla_\theta \log \pi_\theta (s,a)$ which is not always known (when something else than a neural networks is used for the agent's policy).

Direct Policy Search methods using gradient free optimization procedures like Evolution Strategies or Cross Entropy Method (CEM) are interesting alternatives to Policy Gradient algorithms.
They can be successfully applied as long as the $\pi_\theta$ policy has no more than few hundreds of parameters.
Moreover, these method can solve complex problems that cannot be modeled as Markov Decision Processes.

As for previous Reinforcement Learning labs, we will use standard problems provided by Gymnasium suite.
Especially, we will try to solve the LunarLander-v3 problem (https://gymnasium.farama.org/environments/box2d/lunar_lander/) which offers both continuous states and action spaces.

As for previous labs, you can either:
- open, edit and execute the notebook in *Google Colab* following this link: https://colab.research.google.com/github/jeremiedecock/polytechnique-csc-52081-ep-2025-students/blob/main/lab7_optim.ipynb ; this is the **recommended** choice as you have nothing to install on your computer
- open, edit and execute the notebook in *MyBinder* (if for any reason the Google Colab solution doesn't work): https://mybinder.org/v2/gh/jeremiedecock/polytechnique-csc-52081-ep-2025-students/main?filepath=lab7_optim.ipynb
- download, edit and execute the notebook on your computer if Python3 and JypyterLab are already installed: https://github.com/jeremiedecock/polytechnique-csc-52081-ep-2025-students/raw/main/lab7_optim.ipynb

If you work with Google Colab or MyBinder, **remember to save or download your work regularly or you may lose it!**

## Lab Submission

Please submit your completed notebook in [Moodle : "Lab 7 - Submission"](https://moodle.polytechnique.fr/course/section.php?id=66535).

### Submission Guidelines

1. **File Naming:** Rename your notebook as follows: **`firstname_lastname-07.ipynb`** where `firstname` and `lastname` match your email address. *Example: `jesse_read-07.ipynb`*
2. **Clear Output Cells:** To reduce file size (**must be under 500 KB**), clear all output cells before submitting. This includes rendered images, videos, plots, and dataframes...
   - **JupyterLab:**
     - Click **"Kernel" → "Restart Kernel and Clear Outputs of All Cells..."**
     - Then go to **"File" → "Save Notebook As..."**
   - **Google Colab:**
     - Click **"Edit" → "Clear all outputs"**
     - Then go to **"File" → "Download" → "Download.ipynb"**
   - **VSCode:**
     - Click **"Clear All Outputs"**
     - Then **save your file**
3. **Upload Your File:** Only **`.ipynb`** files are accepted.

**Note:** Bonus parts (if any) are optional, as their name suggests.


## Setup the Python environment

This notebook relies on several libraries including `cma`, `gymnasium[box2d,classic-control]` (v1.0.0), `ipywidgets`, `matplotlib`, `moviepy`, `numpy`, `pandas`, `pygame`, `seaborn`, and `tqdm`.
A complete list of dependencies can be found in the following [requirements-lab7.txt](https://raw.githubusercontent.com/jeremiedecock/polytechnique-csc-52081-ep-2025-students/main/requirements-lab7.txt) file.

### If you use Google Colab

If you use Google Colab, execute the next cell to install required libraries.

In [ ]:
import sys
import subprocess


def is_colab() -> bool:
    return "google.colab" in sys.modules


def run_subprocess_command(cmd: str) -> None:
    # run the command
    process = subprocess.Popen(cmd.split(), stdout=subprocess.PIPE)
    # print the output
    for line in process.stdout:
        print(line.decode().strip())


if is_colab():
    run_subprocess_command("apt install swig")
    run_subprocess_command("pip install -r https://raw.githubusercontent.com/jeremiedecock/polytechnique-csc-52081-ep-2025-students/main/requirements-lab7-google-colab.txt")

### If you have downloaded the notebook on your computer and execute it in your own Python environment

To set up the necessary dependencies, run the following commands to establish a [Python virtual environment (venv)](https://docs.python.org/3/library/venv.html) that includes all the essential libraries for this lab.

#### On Posix systems (Linux, MacOSX, WSL, ...)

```bash
python3 -m venv env-lab7
source env-lab7/bin/activate
python3 -m pip install --upgrade pip
python3 -m pip install -r https://raw.githubusercontent.com/jeremiedecock/polytechnique-csc-52081-ep-2025-students/main/requirements-lab7.txt
```

#### On Windows

```bash
python3 -m venv env-lab7
env-lab7\Scripts\activate.bat
python3 -m pip install --upgrade pip
python3 -m pip install -r https://raw.githubusercontent.com/jeremiedecock/polytechnique-csc-52081-ep-2025-students/main/requirements-lab7.txt
```

### Run CSC-52081-EP notebooks locally in a dedicated Docker container

If you are familiar with Docker, an image is available on Docker Hub for this lab:

```bash
docker run -it --rm --user root -p 8888:8888 -e NB_UID=$(id -u) -e NB_GID=$(id -g) -v "${PWD}":/home/jovyan/work jdhp/csc-52081-ep:latest
```

### Import required packages

In [ ]:
import gymnasium as gym
import math
import numpy as np
import pandas as pd
from pathlib import Path
import itertools
import torch
from typing import cast, List, Tuple, Deque, Optional, Callable

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt

import seaborn as sns
from tqdm.notebook import tqdm

In [ ]:
from IPython.display import Video
from ipywidgets import interact

In [ ]:
import warnings

warnings.filterwarnings("ignore", category=UserWarning)

In [ ]:
sns.set_context("talk")

In [ ]:
FIGS_DIR = Path("figs/") / "lab7"       # Where to save figures (.gif or .mp4 files)
PLOTS_DIR = Path("figs/") / "lab7"      # Where to save plots (.png or .svg files)
MODELS_DIR = Path("models/") / "lab7"   # Where to save models (.pth files)

In [ ]:
if not FIGS_DIR.exists():
    FIGS_DIR.mkdir(parents=True)
if not PLOTS_DIR.exists():
    PLOTS_DIR.mkdir(parents=True)
if not MODELS_DIR.exists():
    MODELS_DIR.mkdir(parents=True)

## Define the video selector widget

The `video_selector` function, defined in the next cell, will be used in exercises 2, 3, 4 and 5 to display different episodes of the trained agent.

In [ ]:
def video_selector(file_path: List[Path]) -> Video:
    return Video(file_path, embed=True, html_attributes="controls autoplay loop")

## PyTorch Refresher and Cheat Sheet

In this lab, we will be implementing our deep reinforcement learning algorithms using PyTorch.
If you need a refresher, you might find this [PyTorch Cheat Sheet](https://pytorch.org/tutorials/beginner/ptcheat.html) helpful. It provides a quick reference for many of the most commonly used PyTorch functions and concepts, and can be a valuable resource as you work through this lab.

You can also refer to the [official documentation](https://pytorch.org/docs/stable/index.html).

## PyTorch setup

GPUs are useless for this lab. The following cell ask PyTorch to work on CPU.

In [ ]:
device = "cpu"

## Exercise 1: Implement CEM and test it on the CartPole environment

Before solving the Lunar Lander environment, we will practice on the (simpler) CartPole environment.

**Reminder**: a description of the CartPole environment is available at https://gymnasium.farama.org/environments/classic_control/cart_pole/. This environment offers a continuous state space and discrete action space.

### Implement the policy to optimize

An implementation of the CartPole policy is given in the following cell.

In [ ]:
def sigmoid(x: np.ndarray) -> np.ndarray:
    """
    Compute the sigmoid function for the input array.

    Parameters
    ----------
    x : np.ndarray
        The input array for which to compute the sigmoid function.

    Returns
    -------
    np.ndarray
        The output array with the sigmoid function applied element-wise.
    """
    return 1.0 / (1.0 + np.exp(-x))


# Logistic Regression ############################################


class LogisticRegression:
    """
    Logistic Regression model for binary classification.

    Parameters
    ----------
    observations_size : int
        The size of the observation space.

    Attributes
    ----------
    observation_size : int
        The size of the observation space.
    params : np.ndarray
        The parameters of the logistic regression model.
    """

    def __init__(self, observations_size: int):
        """
        Initialize the LogisticRegression model with random parameters.

        Parameters
        ----------
        observations_size : int
            The size of the observation space.
        """
        self.observation_size = observations_size
        self.params = np.random.rand(observations_size)

    def __call__(self, observation: np.ndarray) -> int:
        """
        Predict the class label for a given observation.

        Parameters
        ----------
        observation : np.ndarray
            The input observation.

        Returns
        -------
        int
            The predicted class label (0 or 1).
        """
        prob_push_right = sigmoid(np.dot(observation, np.transpose(self.params)))
        return 1 if np.random.rand() < prob_push_right else 0

    def get_params(self) -> np.ndarray:
        """
        Get the parameters of the logistic regression model.

        Returns
        -------
        np.ndarray
            The parameters of the logistic regression model.
        """
        return self.params.copy()

    def set_params(self, params: np.ndarray) -> None:
        """
        Set the parameters of the logistic regression model.

        Parameters
        ----------
        params : np.ndarray
            The parameters of the logistic regression model.
        """
        self.params = params.copy()

### Test the untrained agent

In [ ]:
def test_agent(
    env: gym.Env, policy: torch.nn.Module, num_episode: int = 1
) -> List[float]:
    """
    Test a naive agent in the given environment using the provided Q-network.

    Parameters
    ----------
    env : gym.Env
        The environment in which to test the agent.
    policy : torch.nn.Module
        The neural network to use for decision making.
    num_episode : int, optional
        The number of episodes to run, by default 1.

    Returns
    -------
    List[float]
        A list of rewards per episode.
    """
    episode_reward_list = []

    for episode_id in range(num_episode):
        observation, info = env.reset()
        done = False
        episode_reward = 0.0

        while not done:

            action = policy(observation)

            next_observation, reward, terminated, truncated, info = env.step(action)
            done = terminated or truncated

            episode_reward += float(reward)

            observation = next_observation

        episode_reward_list.append(episode_reward)
        print(f"Episode reward: {episode_reward}")

    return episode_reward_list

In [ ]:
VIDEO_PREFIX_EX1_UNTRAINED = "lab7_ex1_untained"

NUM_EPISODES = 3

file_path_list = [
    FIGS_DIR / f"{VIDEO_PREFIX_EX1_UNTRAINED}-episode-{episode_index}.mp4"
    for episode_index in range(NUM_EPISODES)
]

for file_path in file_path_list:
    file_path.unlink(missing_ok=True)

env = gym.make("CartPole-v1", render_mode="rgb_array")
env = gym.wrappers.RecordVideo(
    env,
    video_folder=str(FIGS_DIR),
    name_prefix=VIDEO_PREFIX_EX1_UNTRAINED,
    episode_trigger=lambda x: True,
)
env = gym.wrappers.RecordEpisodeStatistics(env, buffer_length=NUM_EPISODES)

nn_policy = LogisticRegression(env.observation_space.shape[0])

test_agent(env, nn_policy, num_episode=NUM_EPISODES)

# print(f'Episode time taken: {env.time_queue}')
# print(f'Episode total rewards: {env.return_queue}')
# print(f'Episode lengths: {env.length_queue}')

env.close()

print("\nSelect the episode to play here 👇\n")

interact(video_selector, file_path=file_path_list);

### Implement the objective function

Optimization algorithms aim to find the minimum of a function. This function is called an "objective function".
The cell below implements the framework for such a function.
Note that in reinforcement learning, by convention the score is a reward to maximize whereas in mathematical optimization the score is a cost to minimize (most optimization libraries like PyCMA used in this lab impose this convention) ; the objective function will therefore return the opposite of the reward as the score of evaluated policies.

In [ ]:
class ObjectiveFunction:
    """
    Objective function for evaluating a policy in a given environment.

    Parameters
    ----------
    env : gym.Env
        The environment in which to evaluate the policy.
    policy : torch.nn.Module
        The policy to evaluate.
    num_episodes : int, optional
        The number of episodes to run for each evaluation, by default 1.
    max_time_steps : float, optional
        The maximum number of time steps per episode, by default float("inf").
    minimization_solver : bool, optional
        Whether the solver is a minimization solver, by default True.

    Attributes
    ----------
    env : gym.Env
        The environment in which to evaluate the policy.
    policy : torch.nn.Module
        The policy to evaluate.
    num_episodes : int
        The number of episodes to run for each evaluation.
    max_time_steps : float
        The maximum number of time steps per episode.
    minimization_solver : bool
        Whether the solver is a minimization solver.
    num_evals : int
        The number of evaluations performed.
    """

    def __init__(
        self,
        env: gym.Env,
        policy: torch.nn.Module,
        num_episodes: int = 1,
        max_time_steps: float = float("inf"),
        minimization_solver: bool = True,
    ):
        self.env = env
        self.policy = policy
        self.num_episodes = num_episodes
        self.max_time_steps = max_time_steps
        self.minimization_solver = minimization_solver

        self.num_evals = 0

    def eval(self, policy_params: np.ndarray, num_episodes: Optional[int] = None, max_time_steps: Optional[float] = None) -> float:
        """
        Evaluate a policy.

        Parameters
        ----------
        policy_params : np.ndarray
            The parameters of the policy to evaluate.
        num_episodes : int, optional
            The number of episodes to run for each evaluation, by default None.
        max_time_steps : float, optional
            The maximum number of time steps per episode, by default None.

        Returns
        -------
        float
            The average total rewards over the evaluation episodes.
        """
        self.policy.set_params(policy_params)

        self.num_evals += 1

        if num_episodes is None:
            num_episodes = self.num_episodes

        if max_time_steps is None:
            max_time_steps = self.max_time_steps

        average_total_rewards = 0

        for i_episode in range(num_episodes):
            total_rewards = 0.0
            observation, info = self.env.reset()

            for t in range(max_time_steps):
                action = self.policy(observation)
                observation, reward, terminated, truncated, info = self.env.step(action)
                total_rewards += reward

                done = terminated or truncated

                if done:
                    break

            average_total_rewards += float(total_rewards) / num_episodes

        if self.minimization_solver:
            average_total_rewards *= -1.0

        return average_total_rewards  # Optimizers do minimization by default...

    def __call__(self, policy_params: np.ndarray, num_episodes: Optional[int] = None, max_time_steps: Optional[float] = None) -> float:
        """
        Evaluate a policy.

        Parameters
        ----------
        policy_params : np.ndarray
            The parameters of the policy to evaluate.
        num_episodes : int, optional
            The number of episodes to run for each evaluation, by default None.
        max_time_steps : float, optional
            The maximum number of time steps per episode, by default None.

        Returns
        -------
        float
            The average total rewards over the evaluation episodes.
        """
        return self.eval(policy_params, num_episodes, max_time_steps)

### Implement the CEM optimization algorithm

**Task 1**: Implement the `cem_uncorrelated` function that search the best $\theta$ parameters with a Cross Entropy Method. Use the objective function defined above.
$\mathbb{P}$ can be defined as an multivariate normal distribution $\mathcal{N}\left( \boldsymbol{\mu}, \boldsymbol{\sigma^2} \boldsymbol{\Sigma} \right)$ where $\boldsymbol{\mu}$ and $\boldsymbol{\sigma^2} \boldsymbol{\Sigma}$ are vectors i.e. we use one mean and one variance parameters per dimension of $\boldsymbol{\theta}$.

**Cross Entropy**

**Input**:<br>
$\quad\quad$ $f$: the objective function<br>
$\quad\quad$ $\mathbb{P}$: family of distribution<br>
$\quad\quad$ $\boldsymbol{\theta}$: initial parameters for the proposal distribution $\mathbb{P}$<br>

**Algorithm parameter**:<br>
$\quad\quad$ $m$: sample size<br>
$\quad\quad$ $m_{\text{elite}}$: number of samples to use to fit $\boldsymbol{\theta}$<br>

**FOR EACH** iteration<br>
$\quad\quad$ samples $\leftarrow \{ \boldsymbol{x}_1, \dots, \boldsymbol{x}_m \}$ with $\boldsymbol{x}_i \sim \mathbb{P}(\boldsymbol{\theta}) ~~ \forall i \in 1\dots m$<br>
$\quad\quad$ elite $\leftarrow $ { $m_{\text{elite}}$ best samples } $\quad$ (i.e. select best samples according to $f$)<br>
$\quad\quad$ $\boldsymbol{\theta} \leftarrow $ fit $\mathbb{P}(\boldsymbol{\theta})$ to the elite samples<br>

**RETURN** $\boldsymbol{\theta}$

In [ ]:
def cem_uncorrelated(
    objective_function: Callable[[np.ndarray], float],
    mean_array: np.ndarray,
    var_array: np.ndarray,
    max_iterations: int = 500,
    sample_size: int = 50,
    elite_frac: float = 0.2,
    print_every: int = 10,
    success_score: float = float("inf"),
    num_evals_for_stop: Optional[int] = None,
    hist_dict: Optional[dict] = None,
) -> np.ndarray:
    """
    Cross-entropy method.

    Parameters
    ----------
    objective_function : Callable[[np.ndarray], float]
        The function to maximize.
    mean_array : np.ndarray
        The initial proposal distribution (mean vector).
    var_array : np.ndarray
        The initial proposal distribution (variance vector).
    max_iterations : int, optional
        Number of training iterations, by default 500.
    sample_size : int, optional
        Size of population at each iteration, by default 50.
    elite_frac : float, optional
        Rate of top performers to use in update with elite_frac ∈ ]0;1], by default 0.2.
    print_every : int, optional
        How often to print average score, by default 10.
    success_score : float, optional
        The score at which to stop the optimization, by default float("inf").
    num_evals_for_stop : Optional[int], optional
        Number of evaluations for stopping criteria, by default None.
    hist_dict : Optional[dict], optional
        Dictionary to log the history, by default None.

    Returns
    -------
    np.ndarray
        The optimized mean vector.
    """
    assert 0.0 < elite_frac <= 1.0

    n_elite = math.ceil(sample_size * elite_frac)

    for iteration_index in range(0, max_iterations):

        # SAMPLE A NEW POPULATION OF SOLUTIONS (X VECTORS) ####################
        x_array = np.random.multivariate_normal(mean_array, np.diag(var_array), sample_size)

        # EVALUATE SAMPLES AND EXTRACT THE BEST ONES ("ELITE") ################
        score_array = np.array([objective_function(x) for x in x_array])

        sorted_indices_array = np.argsort(score_array)  # Sort from the lower score to the higher one
        elite_indices_array = sorted_indices_array[:n_elite]  # Recall: we *minimize* the objective function thus we take the samples that are at the beginning of the sorted_indices

        elite_x_array = x_array[elite_indices_array]

        # FIT THE NORMAL DISTRIBUTION ON THE ELITE POPULATION #################
        mean_array = np.mean(elite_x_array, axis=0)
        var_array = np.var(elite_x_array, axis=0)
        score = np.mean(score_array[elite_indices_array])

        # PRINT STATUS ########################################################
        if iteration_index % print_every == 0:
            print("Iteration {}\tScore {}".format(iteration_index, score))

        if hist_dict is not None:
            hist_dict[iteration_index] = [score] + mean_array.tolist() + var_array.tolist()

        # STOPPING CRITERIA ####################################################
        if num_evals_for_stop is not None:
            score = np.mean(score_array[:num_evals_for_stop])

        # `num_evals_for_stop = None` may be used to fasten computations but it introduces bias...
        if score <= success_score:
            break

    return mean_array


### Train the agent

**Task 2:** train your implementation using the following cells.

In [ ]:
env = gym.make("CartPole-v1")

nn_policy = LogisticRegression(env.observation_space.shape[0])

objective_function = ObjectiveFunction(
    env=env, policy=nn_policy, num_episodes=10, max_time_steps=1000
)

In [ ]:
%%time

hist_dict = {}

num_params = len(nn_policy.get_params())

init_mean_array = np.random.random(num_params)
init_var_array = np.ones(num_params) * 100.0

optimized_policy_params = cem_uncorrelated(
    objective_function=objective_function,
    mean_array=init_mean_array,
    var_array=init_var_array,
    max_iterations=30,
    sample_size=50,
    elite_frac=0.1,
    print_every=1,
    success_score=-500,
    num_evals_for_stop=None,
    hist_dict=hist_dict,
)

env.close()

In [ ]:
df = pd.DataFrame.from_dict(
    hist_dict,
    orient="index",
    columns=["score", "mu1", "mu2", "mu3", "mu4", "var1", "var2", "var3", "var4"],
)
ax = df.score.plot(title="Average reward", figsize=(20, 5))
plt.xlabel("Training Steps")
plt.ylabel("Reward")
plt.savefig(PLOTS_DIR / "lab7_ex1_cem_cartpole_avg_reward_wrt_iterations.png");

In [ ]:
ax = df[["mu1", "mu2", "mu3", "mu4"]].plot(
    title="Theta w.r.t training steps", figsize=(20, 5)
);
plt.xlabel("Training Steps")
plt.savefig(PLOTS_DIR / "lab7_ex1_cem_cartpole_params_wrt_iterations.png");

In [ ]:
ax = df[["var1", "var2", "var3", "var4"]].plot(
    logy=True, title="Variance w.r.t training steps", figsize=(20, 5)
)
plt.xlabel("Training Steps")
plt.savefig(PLOTS_DIR / "lab7_ex1_cem_cartpole_var_wrt_iterations.png");

In [ ]:
print("Optimized weights: ", optimized_policy_params)

### Test the trained agent

**Task 3:** test the optimized policy

In [ ]:
VIDEO_PREFIX_EX1_CEM_CARTPOLE_TRAINED = "lab7_ex1_cem_cartpole_tained"

NUM_EPISODES = 3

file_path_list = [
    FIGS_DIR / f"{VIDEO_PREFIX_EX1_CEM_CARTPOLE_TRAINED}-episode-{episode_index}.mp4"
    for episode_index in range(NUM_EPISODES)
]

for file_path in file_path_list:
    file_path.unlink(missing_ok=True)

env = gym.make("CartPole-v1", render_mode="rgb_array")
env = gym.wrappers.RecordVideo(
    env,
    video_folder=str(FIGS_DIR),
    name_prefix=VIDEO_PREFIX_EX1_CEM_CARTPOLE_TRAINED,
    episode_trigger=lambda x: True,
)
env = gym.wrappers.RecordEpisodeStatistics(env, buffer_length=NUM_EPISODES)

objective_function = ObjectiveFunction(env=env, policy=nn_policy)

objective_function.eval(optimized_policy_params, num_episodes=NUM_EPISODES, max_time_steps=200)

# print(f'Episode time taken: {env.time_queue}')
# print(f'Episode total rewards: {env.return_queue}')
# print(f'Episode lengths: {env.length_queue}')

env.close()

print("\nSelect the episode to play here 👇\n")

interact(video_selector, file_path=file_path_list);

## Exercise 2: implement SAES and solve the CartPole problem with it

### Implement the (1+1)-SA-ES optimization algorithm

**Task 1**: Implement the `saes_1_1` function that search the best $\theta$ parameters with a (1+1)-SA-ES algorithm. Use the objective function defined above.

**(1+1)-SA-ES**

**Input**:<br>
$\quad\quad$ $f$: the objective function<br>
$\quad\quad$ $\boldsymbol{x}$: initial solution<br>

**Algorithm parameter**:<br>
$\quad\quad$ $\tau$: self-adaptation learning rate<br>

**FOR EACH** generation<br>
$\quad\quad$ 1. mutation of $\sigma$ (current individual strategy) : $\sigma' \leftarrow \sigma ~ e^{\tau \mathcal{N}(0,1)}$<br>
$\quad\quad$ 2. mutation of $\boldsymbol{x}$ (current solution) : $\boldsymbol{x}' \leftarrow \boldsymbol{x} + \sigma' ~ \mathcal{N}(0,1)$<br>
$\quad\quad$ 3. eval $f(\boldsymbol{x}')$<br>
$\quad\quad$ 4. survivor selection $\boldsymbol{x} \leftarrow \boldsymbol{x}'$ and $\sigma \leftarrow \sigma'$ if $f(\boldsymbol{x}') \leq f(\boldsymbol{x})$<br>

**RETURN** $\boldsymbol{x}$

In [ ]:
def saes_1_1(
    objective_function: Callable[[np.ndarray], float],
    x_array: np.ndarray,
    sigma_array: np.ndarray,
    max_iterations: int = 500,
    tau: Optional[float] = None,
    print_every: int = 10,
    success_score: float = float("inf"),
    num_evals_for_stop: Optional[int] = None,
    hist_dict: Optional[dict] = None,
) -> np.ndarray:
    """
    (1+1)-Self-Adaptive Evolution Strategy (SA-ES) optimization algorithm.

    Parameters
    ----------
    objective_function : Callable[[np.ndarray], float]
        The function to minimize.
    x_array : np.ndarray
        The initial solution vector.
    sigma_array : np.ndarray
        The initial strategy parameter vector (step sizes).
    max_iterations : int, optional
        The maximum number of iterations, by default 500.
    tau : Optional[float], optional
        The self-adaptation learning rate, by default None.
    print_every : int, optional
        How often to print the current score, by default 10.
    success_score : float, optional
        The score at which to stop the optimization, by default float("inf").
    num_evals_for_stop : Optional[int], optional
        Number of evaluations for stopping criteria, by default None.
    hist_dict : Optional[dict], optional
        Dictionary to log the history, by default None.

    Returns
    -------
    np.ndarray
        The optimized solution vector.
    """
    # Number of dimension of the solution space
    d = x_array.shape[0]

    if tau is None:
        # Self-adaptation learning rate
        tau = 1.0 / (2.0 * d)

    score = objective_function(x_array)

    for iteration_index in range(0, max_iterations):
        # 1. Mutation of sigma (current "individual strategy")
        new_sigma_array = sigma_array * np.exp(tau * np.random.normal(size=d))

        # 2. Mutation of x (current solution)
        new_x_array = x_array + new_sigma_array * np.random.normal(size=d)

        # 3. Eval f(x')
        new_score = objective_function(new_x_array)

        # 4. survivor selection (we follow the ES convention and do minimization)
        # You may try `new_score < score` for less exploration
        if new_score < score:  
            score = new_score
            x_array = new_x_array
            sigma_array = new_sigma_array

        # PRINT STATUS ########################################################

        if iteration_index % print_every == 0:
            print("Iteration {}\tScore {}".format(iteration_index, score))

        if hist_dict is not None:
            hist_dict[iteration_index] = [score] + x_array.tolist() + sigma_array.tolist()

        # STOPPING CRITERIA ####################################################

        if num_evals_for_stop is not None:
            score = np.mean([objective_function(x_array) for _ in range(num_evals_for_stop)])

        # `num_evals_for_stop = None` may be used to fasten computations but it introduces bias...
        if score <= success_score:
            break

    return x_array

### Train the agent

**Task 2:** train your implementation using the following cells.

In [ ]:
env = gym.make("CartPole-v1")

nn_policy = LogisticRegression(env.observation_space.shape[0])

objective_function = ObjectiveFunction(
    env=env, policy=nn_policy, num_episodes=10, max_time_steps=1000
)

In [ ]:
%%time

hist_dict = {}

num_params = len(nn_policy.get_params())

initial_solution_array = np.random.random(num_params)
initial_sigma_array = np.ones(num_params) * 1.0

optimized_policy_params = saes_1_1(
    objective_function=objective_function,
    x_array=initial_solution_array,
    sigma_array=initial_sigma_array,
    tau=0.001,
    max_iterations=1000,
    print_every=100,
    success_score=-500,
    num_evals_for_stop=None,
    hist_dict=hist_dict,
)

env.close()

In [ ]:
df = pd.DataFrame.from_dict(
    hist_dict,
    orient="index",
    columns=[
        "score",
        "mu1",
        "mu2",
        "mu3",
        "mu4",
        "sigma1",
        "sigma2",
        "sigma3",
        "sigma4",
    ],
)
ax = df.score.plot(title="Average reward", figsize=(30, 5))
plt.xlabel("Training Steps")
plt.ylabel("Reward")
plt.savefig(PLOTS_DIR / "lab7_ex2_saes_cartpole_avg_reward_wrt_iterations.png");

In [ ]:
ax = df[["mu1", "mu2", "mu3", "mu4"]].plot(
    title="Theta w.r.t training steps", figsize=(30, 5)
)
plt.xlabel("Training Steps")
plt.savefig(PLOTS_DIR / "lab7_ex2_saes_cartpole_params_wrt_iterations.png");

In [ ]:
ax = df[["sigma1", "sigma2", "sigma3", "sigma4"]].plot(
    logy=True, title="Sigma w.r.t training steps", figsize=(30, 5)
)
plt.xlabel("Training Steps")
plt.savefig(PLOTS_DIR / "lab7_ex2_saes_cartpole_var_wrt_iterations.png");

In [ ]:
print("Optimized weights: ", optimized_policy_params)

### Test the trained agent

**Task 3:** test the optimized policy

In [ ]:
VIDEO_PREFIX_EX2_SAES_CARTPOLE_TRAINED = "lab7_ex2_saes_cartpole_tained"

NUM_EPISODES = 3

file_path_list = [
    FIGS_DIR / f"{VIDEO_PREFIX_EX2_SAES_CARTPOLE_TRAINED}-episode-{episode_index}.mp4"
    for episode_index in range(NUM_EPISODES)
]

for file_path in file_path_list:
    file_path.unlink(missing_ok=True)

env = gym.make("CartPole-v1", render_mode="rgb_array")
env = gym.wrappers.RecordVideo(
    env,
    video_folder=str(FIGS_DIR),
    name_prefix=VIDEO_PREFIX_EX2_SAES_CARTPOLE_TRAINED,
    episode_trigger=lambda x: True,
)
env = gym.wrappers.RecordEpisodeStatistics(env, buffer_length=NUM_EPISODES)

objective_function = ObjectiveFunction(env=env, policy=nn_policy)

objective_function.eval(optimized_policy_params, num_episodes=NUM_EPISODES, max_time_steps=200)

# print(f'Episode time taken: {env.time_queue}')
# print(f'Episode total rewards: {env.return_queue}')
# print(f'Episode lengths: {env.length_queue}')

env.close()

print("\nSelect the episode to play here 👇\n")

interact(video_selector, file_path=file_path_list);

**Task 4:** try different values of $\tau$. What happen ?

## Exercise 3: Implement a parametric policy $\pi_\theta$ for environments having a continuous action space

To solve problems having a continuous space, especially to solve the LunarLander problem in the next exercise, we need to define and implement an appropriate parametric policy.
For this purpose, we recommend the following neural network:
- one hidden layer of 16 units having a ReLu activation function
- a tanh activation function on the output layer (be careful on the number of output units)

To solve environments with continuous action space like LunarLander with Direct Policy Search methods, a simple procedure that compute the feed forward signal is needed (we don't do back propagation here).
A procedure to set/get weights of the network from/to a single vector $\theta$ will also be required.

In [ ]:
class NeuralNetworkPolicy(torch.nn.Module):
    """
    A neural network used as a policy.

    Attributes
    ----------
    hidden_layer : torch.nn.Linear
        The hidden layer of the neural network.
    output_layer : torch.nn.Linear
        The output layer of the neural network.

    Methods
    -------
    forward(observation: torch.Tensor) -> np.ndarray
        Define the forward pass of the NeuralNetworkPolicy.
    get_params() -> np.ndarray
        Get the parameters of the neural network.
    set_params(params: np.ndarray) -> None
        Set the parameters of the neural network.
    """

    def __init__(
        self, observations_size: int, actions_size: int, hidden_size: int = 16
    ):
        """
        Initialize a new instance of NeuralNetworkPolicy.

        Parameters
        ----------
        observations_size : int
            The size of the observation space (i.e. the number of features i.e. the number of elements in the observation vector).
        actions_size : int
            The size of the action space.
        hidden_size : int, optional
            The number of units in the hidden layer, by default 16.
        """
        super(NeuralNetworkPolicy, self).__init__()

        self.hidden_layer = torch.nn.Linear(observations_size, hidden_size)
        self.output_layer = torch.nn.Linear(hidden_size, actions_size)

    def forward(self, observation: torch.Tensor) -> np.ndarray:
        """
        Calculate the probability of action a_1 for the given observation.

        Parameters
        ----------
        observation : torch.Tensor
            The input tensor (observation).
            The shape of the tensor should be (N, dim),
            where N is the number of observations vectors in the batch
            and dim is the dimension of observation vectors.

        Returns
        -------
        np.ndarray
            The output tensor (the probability of each action for the given observation).
        """
        with torch.no_grad():
            observation_tensor = torch.tensor(observation, dtype=torch.float32, device=device)
            hidden_tensor = torch.nn.functional.relu(self.hidden_layer(observation_tensor))
            output_tensor = torch.tanh(self.output_layer(hidden_tensor))
            return output_tensor.squeeze().detach().cpu().numpy()

    def get_params(self) -> np.ndarray:
        """
        Get the parameters of the neural network.

        Returns
        -------
        np.ndarray
            The parameters of the neural network.
        """
        params_tensor = torch.nn.utils.parameters_to_vector(self.parameters())
        return params_tensor.detach().cpu().numpy()

    def set_params(self, params: np.ndarray) -> None:
        """
        Set the parameters of the neural network.

        Parameters
        ----------
        params : np.ndarray
            The parameters of the neural network.
        """
        params_tensor = torch.tensor(params, dtype=torch.float32, device=device)
        torch.nn.utils.vector_to_parameters(params_tensor, self.parameters())

## Exercise 4: solve the LunarLander problem (continuous version) with CEM and SAES

### Hands on the LunarLander problem

**Task 1:** read https://gymnasium.farama.org/environments/box2d/lunar_lander/ to discover the LunarLanderContinuous environment.

**Notice:** A reminder of Gymnasium main concepts is available at https://gymnasium.farama.org/introduction/basic_usage/.

Print some information about the environment:

In [ ]:
%%capture
%pip install swig
%pip install gymnasium[box2d]
%pip install Box2D

In [ ]:
env = gym.make("LunarLander-v3", continuous=True)
print("Observation space dimension is:", env.observation_space.shape[0])
print("Observation upper bounds:", env.observation_space.high)
print("Observation lower bounds:", env.observation_space.low)
print("Actions upper bounds:", env.action_space.high)
print("Actions lower bounds:", env.action_space.low)
env.close()

**Task 2:** Run the following cells and check different basic policies (for instance constant actions or randomly drawn actions) to discover the Lunar Lander environment.

In [ ]:
VIDEO_PREFIX_EX4_UNTRAINED = "lab7_ex4_lunarlander_untrained"

(FIGS_DIR / f"{VIDEO_PREFIX_EX4_UNTRAINED}-episode-0.mp4").unlink(missing_ok=True)

env = gym.make("LunarLander-v3", continuous=True, render_mode="rgb_array")
env = gym.wrappers.RecordVideo(env, video_folder=str(FIGS_DIR), name_prefix=VIDEO_PREFIX_EX4_UNTRAINED)

observation, info = env.reset()
done = False

while not done:
    # action = np.array([1., 1.])
    action = np.array([-1.0, -1.0])
    # action = env.action_space.sample()   # Random policy

    observation, reward, terminated, truncated, info = env.step(action)
    done = terminated or truncated

env.close()

Video(
    FIGS_DIR / f"{VIDEO_PREFIX_EX4_UNTRAINED}-episode-0.mp4",
    embed=True,
    html_attributes="controls autoplay loop",
)

**Note**: If you encounter a dependency issue with *swig*, follow the procedure outlined at https://gymnasium.farama.org/environments/box2d/

**Question 1:** We want to use CEM and SAES to compute the optimal policy for the Lunar Lander environment.
What is the size of the search space (number of dimensions) for optimizers knowing that the policy is the one defined in exercise 3 (a neural network of one hidden layer of 16 neurons) and knowing that the observation space of the Lunar Lander environment is $\mathcal{S} = \mathbb{R}^8$ and its action space is $\mathcal{A} \subset \mathbb{R}^2$ ?

The size of the seach space (number of dimensions) for the optimizers is the total number of parameters in the neural network policy that needs to be optimized. The NN architecture is composed by:

**1.** Input layer of 8 neurons (corresponding to the 8 dimentions of the observation state).

**2.** Hidden layer of 16 neurons.

**3.** Output layer of 2 neurons (corresponding to the 2 dimentions of the action space).

With that, the number of parameters is:

**Weights between Input layer and Hidden layer:** 8 * 16 = 128

**Biases for Hidden layer:** 16

**Weights between Hidden layer and Output layer:** 16 * 2 = 32

**Biases for Output layer:** 2

Therefore, in total, we have 128 + 16 + 32 + 2 = **178** parameters to optimize.

### Train the CEM agent on the LunarLander environment

**Task 3:** Train the agent using the CEM algorithm

In [ ]:
env = gym.make("LunarLander-v3", continuous=True)

nn_policy = NeuralNetworkPolicy(
    observations_size=env.observation_space.shape[0],
    actions_size=env.action_space.shape[0],
    hidden_size=16,
)

objective_function = ObjectiveFunction(
    env=env, policy=nn_policy, num_episodes=3, max_time_steps=500
)

In [ ]:
%%time

hist_dict = {}

num_params = len(nn_policy.get_params())

init_mean_array = np.random.random(num_params)
init_var_array = np.ones(num_params) * 1000.0

optimized_policy_params = cem_uncorrelated(
    objective_function=objective_function,
    mean_array=init_mean_array,
    var_array=init_var_array,
    max_iterations=100,
    sample_size=50,
    elite_frac=0.2,
    print_every=10,
    success_score=-200,
    hist_dict=hist_dict,
)

objective_function.env.close()

In [ ]:
df = pd.DataFrame.from_dict(hist_dict, orient="index")
ax = df.iloc[:, 0].plot(title="Average reward", figsize=(20, 5))
plt.xlabel("Training Steps")
plt.ylabel("Reward")
plt.savefig(PLOTS_DIR / "lab7_ex4_cem_lunarlander_avg_reward_wrt_iterations.png");

In [ ]:
ax = df.iloc[:, 1:66].plot(
    title="Theta w.r.t training steps", legend=None, figsize=(20, 10)
)
# ax.get_legend().remove()
plt.xlabel("Training Steps")
plt.savefig(PLOTS_DIR / "lab7_ex4_cem_lunarlander_params_wrt_iterations.png");

In [ ]:
ax = df.iloc[:, 67:].plot(
    logy=True, title="Variance w.r.t training steps", legend=None, figsize=(20, 10)
)
# ax.get_legend().remove()
plt.xlabel("Training Steps")
plt.savefig(PLOTS_DIR / "lab7_ex4_cem_lunarlander_var_wrt_iterations.png");

In [ ]:
print("Optimized weights: ", optimized_policy_params)

### Test the trained agent

**Task 4:** check the optimized policy.

In [ ]:
VIDEO_PREFIX_EX4_CEM_LUNARLANDER_TRAINED = "lab7_ex4_cem_lunarlander_tained"

NUM_EPISODES = 3

file_path_list = [
    FIGS_DIR / f"{VIDEO_PREFIX_EX4_CEM_LUNARLANDER_TRAINED}-episode-{episode_index}.mp4"
    for episode_index in range(NUM_EPISODES)
]

for file_path in file_path_list:
    file_path.unlink(missing_ok=True)

env = gym.make("LunarLander-v3", continuous=True, render_mode="rgb_array")
env = gym.wrappers.RecordVideo(
    env,
    video_folder=str(FIGS_DIR),
    name_prefix=VIDEO_PREFIX_EX4_CEM_LUNARLANDER_TRAINED,
    episode_trigger=lambda x: True,
)
env = gym.wrappers.RecordEpisodeStatistics(env, buffer_length=NUM_EPISODES)

objective_function = ObjectiveFunction(env=env, policy=nn_policy)

objective_function.eval(optimized_policy_params, num_episodes=NUM_EPISODES, max_time_steps=500)

# print(f'Episode time taken: {env.time_queue}')
# print(f'Episode total rewards: {env.return_queue}')
# print(f'Episode lengths: {env.length_queue}')

env.close()

print("\nSelect the episode to play here 👇\n")

interact(video_selector, file_path=file_path_list);

### Train the (1+1)-SA-ES agent on the LunarLander environment

**Task 5:** Train the agent using the (1+1)-SA-ES algorithm

In [ ]:
env = gym.make("LunarLander-v3", continuous=True)

nn_policy = NeuralNetworkPolicy(
    observations_size=env.observation_space.shape[0],
    actions_size=env.action_space.shape[0],
    hidden_size=16,
)

objective_function = ObjectiveFunction(
    env=env,
    policy=nn_policy,
    num_episodes=5,  # <- resampling
    max_time_steps=500,
)

In [ ]:
%%time

hist_dict = {}

num_params = len(nn_policy.get_params())

initial_solution_array = np.random.random(num_params)
initial_sigma_array = np.ones(num_params) * 1.0

optimized_policy_params = saes_1_1(
    objective_function=objective_function,
    x_array=initial_solution_array,
    sigma_array=initial_sigma_array,
    max_iterations=1000,
    print_every=100,
    success_score=-200,
    num_evals_for_stop=None,
    hist_dict=hist_dict,
)

objective_function.env.close()

In [ ]:
df = pd.DataFrame.from_dict(hist_dict, orient="index")
ax = df.iloc[:, 0].plot(title="Average reward", figsize=(20, 5))
plt.xlabel("Training Steps")
plt.ylabel("Reward")
plt.savefig(PLOTS_DIR / "lab7_ex4_saes_lunarlander_avg_reward_wrt_iterations.png");

In [ ]:
ax = df.iloc[:, 1:66].plot(
    title="Theta w.r.t training steps", legend=None, figsize=(20, 10)
)
# ax.get_legend().remove()
plt.xlabel("Training Steps")
plt.savefig(PLOTS_DIR / "lab7_ex4_saes_lunarlander_params_wrt_iterations.png");

In [ ]:
ax = df.iloc[:, 67:].plot(
    logy=True, title="Variance w.r.t training steps", legend=None, figsize=(20, 10)
)
# ax.get_legend().remove()
plt.xlabel("Training Steps")
plt.savefig(PLOTS_DIR / "lab7_ex4_saes_lunarlander_var_wrt_iterations.png");

In [ ]:
print("Optimized weights: ", optimized_policy_params)

### Test the trained agent

**Task 6:** check the optimized policy.

In [ ]:
VIDEO_PREFIX_EX4_SAES_LUNARLANDER_TRAINED = "lab7_ex4_saes_lunarlander_tained"

NUM_EPISODES = 3

file_path_list = [
    FIGS_DIR / f"{VIDEO_PREFIX_EX4_SAES_LUNARLANDER_TRAINED}-episode-{episode_index}.mp4"
    for episode_index in range(NUM_EPISODES)
]

for file_path in file_path_list:
    file_path.unlink(missing_ok=True)

env = gym.make("LunarLander-v3", continuous=True, render_mode="rgb_array")
env = gym.wrappers.RecordVideo(
    env,
    video_folder=str(FIGS_DIR),
    name_prefix=VIDEO_PREFIX_EX4_SAES_LUNARLANDER_TRAINED,
    episode_trigger=lambda x: True,
)
env = gym.wrappers.RecordEpisodeStatistics(env, buffer_length=NUM_EPISODES)

objective_function = ObjectiveFunction(env=env, policy=nn_policy)

objective_function.eval(optimized_policy_params, num_episodes=NUM_EPISODES, max_time_steps=500)

# print(f'Episode time taken: {env.time_queue}')
# print(f'Episode total rewards: {env.return_queue}')
# print(f'Episode lengths: {env.length_queue}')

env.close()

print("\nSelect the episode to play here 👇\n")

interact(video_selector, file_path=file_path_list);

## Bonus exercise 1: implement CEM with an alternative *Proposal distribution*

Implement CEM with the following alternative *Proposal distribution*: a multivariate normal distribution parametrized by a mean vector and **a covariance matrix** (to use correlations in the search space dimensions).

Test it on the CartPole and the LunarLander environments.

In [ ]:
def cem_correlated(
    objective_function: Callable[[np.ndarray], float],
    mean_array: np.ndarray,
    var_array: np.ndarray,
    max_iterations: int = 500,
    sample_size: int = 50,
    elite_frac: float = 0.2,
    print_every: int = 10,
    success_score: float = float("inf"),
    num_evals_for_stop: Optional[int] = None,
    hist_dict: Optional[dict] = None,
) -> np.ndarray:
    """Cross-entropy method.

    Params
    ======
        objective_function (function): the function to maximize
        mean_array (array of floats): the initial proposal distribution (mean vector)
        var_array (array of floats): the initial proposal distribution (variance vector)
        max_iterations (int): number of training iterations
        sample_size (int): size of population at each iteration
        elite_frac (float): rate of top performers to use in update with elite_frac ∈ ]0;1]
        print_every (int): how often to print average score
        hist_dict (dict): logs
    """
    assert 0.0 < elite_frac <= 1.0

    n_elite = math.ceil(sample_size * elite_frac)

    cov_array = np.diag(var_array)

    for iteration_index in range(0, max_iterations):
        # SAMPLE A NEW POPULATION OF SOLUTIONS (X VECTORS) ####################
        x_array = np.random.multivariate_normal(mean_array, cov_array, sample_size)

        # EVALUATE SAMPLES AND EXTRACT THE BEST ONES ("ELITE") ################
        score_array = np.array([objective_function(x) for x in x_array])

        sorted_indices_array = np.argsort(score_array)  # Sort from the lower score to the higher one
        elite_indices_array = sorted_indices_array[:n_elite]  # Recall: we *minimize* the objective function thus we take the samples that are at the beginning of the sorted_indices

        elite_x_array = x_array[elite_indices_array]

        # FIT THE NORMAL DISTRIBUTION ON THE ELITE POPULATION #################
        mean_array = np.mean(elite_x_array, axis=0)
        cov_array = np.cov(elite_x_array, rowvar=False)
        score = np.mean(score_array[elite_indices_array])

        # PRINT STATUS ########################################################
        if iteration_index % print_every == 0:
            print("Iteration {}\tScore {}".format(iteration_index, score))

        if hist_dict is not None:
            hist_dict[iteration_index] = [score] + mean_array.tolist() + cov_array.flatten().tolist()

        # STOPPING CRITERIA ####################################################
        if num_evals_for_stop is not None:
            score = np.mean(score_array[:num_evals_for_stop])

        # `num_evals_for_stop = None` may be used to fasten computations but it introduces bias...
        if score <= success_score:
            break

    return mean_array


### Test on CartPole

In [ ]:
env = gym.make("CartPole-v1")

nn_policy = LogisticRegression(env.observation_space.shape[0])

objective_function = ObjectiveFunction(
    env=env, policy=nn_policy, num_episodes=10, max_time_steps=1000
)

In [ ]:
%%time

hist_dict = {}

num_params = len(nn_policy.get_params())

init_mean_array = np.random.random(num_params)
init_var_array = np.ones(num_params) * 100.0

optimized_policy_params = cem_correlated(
    objective_function=objective_function,
    mean_array=init_mean_array,
    var_array=init_var_array,
    max_iterations=30,
    sample_size=50,
    elite_frac=0.1,
    print_every=1,
    success_score=-500,
    num_evals_for_stop=None,
    hist_dict=hist_dict,
)

objective_function.env.close()

In [ ]:
df = pd.DataFrame.from_dict(
    hist_dict,
    orient="index",
    columns=["score"]
    + [f"mu{i}" for i in range(1, 5)]
    + [f"cov{i}{j}" for i in range(1, 5) for j in range(1, 5)],
)
ax = df.score.plot(title="Average reward", figsize=(20, 5))
plt.xlabel("Training Steps")
plt.ylabel("Reward")
plt.savefig(PLOTS_DIR / "lab7_ex5_cem_cartpole_avg_reward_wrt_iterations.png");

In [ ]:
ax = df[["mu1", "mu2", "mu3", "mu4"]].plot(
    title="Theta w.r.t training steps", figsize=(20, 5)
)
plt.xlabel("Training Steps")
plt.savefig(PLOTS_DIR / "lab7_ex5_cem_cartpole_params_wrt_iterations.png");

In [ ]:
ax = df[[f"cov{i}{j}" for i in range(1, 5) for j in range(1, 5)]].plot(
    logy=True, title="Variance w.r.t training steps", figsize=(20, 5)
)
plt.xlabel("Training Steps")
plt.savefig(PLOTS_DIR / "lab7_ex5_cem_cartpole_var_wrt_iterations.png");

In [ ]:
print("Optimized weights: ", optimized_policy_params)

In [ ]:
VIDEO_PREFIX_EX5_CEM_CARTPOLE_TRAINED = "lab7_ex5_cem_cartpole_tained"

NUM_EPISODES = 3

file_path_list = [
    FIGS_DIR / f"{VIDEO_PREFIX_EX5_CEM_CARTPOLE_TRAINED}-episode-{episode_index}.mp4"
    for episode_index in range(NUM_EPISODES)
]

for file_path in file_path_list:
    file_path.unlink(missing_ok=True)

env = gym.make("CartPole-v1", render_mode="rgb_array")
env = gym.wrappers.RecordVideo(
    env,
    video_folder=str(FIGS_DIR),
    name_prefix=VIDEO_PREFIX_EX5_CEM_CARTPOLE_TRAINED,
    episode_trigger=lambda x: True,
)
env = gym.wrappers.RecordEpisodeStatistics(env, buffer_length=NUM_EPISODES)

objective_function = ObjectiveFunction(env=env, policy=nn_policy)

objective_function.eval(optimized_policy_params, num_episodes=NUM_EPISODES, max_time_steps=500)

env.close()

# print(f'Episode time taken: {env.time_queue}')
# print(f'Episode total rewards: {env.return_queue}')
# print(f'Episode lengths: {env.length_queue}')

print("\nSelect the episode to play here 👇\n")

interact(video_selector, file_path=file_path_list);

## Bonus exercise 2: test the CMAES algorithm

PyCMA: Python implementation of CMA-ES (from Nikolaus Hansen - CMAP).

Source code:

- http://cma.gforge.inria.fr/cmaes_sourcecode_page.html#python
- https://github.com/CMA-ES/pycma
- https://pypi.org/project/cma/

Official documentation:

- http://cma.gforge.inria.fr/apidocs-pycma/

In [ ]:
import cma

In [ ]:
env = gym.make("LunarLander-v3", continuous=True)

nn_policy = NeuralNetworkPolicy(
    observations_size=env.observation_space.shape[0],
    actions_size=env.action_space.shape[0],
    hidden_size=16,
)

objective_function = ObjectiveFunction(
    env=env, policy=nn_policy, num_episodes=1, max_time_steps=500
)

In [ ]:
%%time

num_params = len(nn_policy.get_params())

x_optimal, es = cma.fmin2(
    objective_function,
    x0=np.random.random(num_params),
    sigma0=10.0,
    options={"maxfevals": 1500},
)
optimized_policy_params = x_optimal

In [ ]:
optimized_policy_params

In [ ]:
sns.set_context("talk")
plt.rcParams["figure.figsize"] = 20, 10

cma.plot();  # shortcut for es.logger.plot()

Test the final policy

In [ ]:
VIDEO_PREFIX_EX6_CMAES_LUNARLANDER_TRAINED = "lab7_ex6_cmaes_lunarlander_tained"

NUM_EPISODES = 3

file_path_list = [
    FIGS_DIR / f"{VIDEO_PREFIX_EX6_CMAES_LUNARLANDER_TRAINED}-episode-{episode_index}.mp4"
    for episode_index in range(NUM_EPISODES)
]

for file_path in file_path_list:
    file_path.unlink(missing_ok=True)

env = gym.make("LunarLander-v3", continuous=True, render_mode="rgb_array")
env = gym.wrappers.RecordVideo(
    env,
    video_folder=str(FIGS_DIR),
    name_prefix=VIDEO_PREFIX_EX6_CMAES_LUNARLANDER_TRAINED,
    episode_trigger=lambda x: True,
)
env = gym.wrappers.RecordEpisodeStatistics(env, buffer_length=NUM_EPISODES)

objective_function = ObjectiveFunction(env=env, policy=nn_policy)

objective_function.eval(optimized_policy_params, num_episodes=NUM_EPISODES, max_time_steps=500)

env.close()

# print(f'Episode time taken: {env.time_queue}')
# print(f'Episode total rewards: {env.return_queue}')
# print(f'Episode lengths: {env.length_queue}')

print("\nSelect the episode to play here 👇\n")

interact(video_selector, file_path=file_path_list);